## Acoustic Integration - Ceiling Sail Orientation

http://localhost:8888/?token=sloth

In [ ]:
if "mi" not in vars():
    from tqdm import trange

    # import torch
    import numpy as np
    import matplotlib.pyplot as plt

    import drjit as dr
    import mitsuba as mi

    from libs import utils

    plt.style.use('ggplot')

    mi.set_variant('cuda_ad_acoustic')
    mi.set_log_level(mi.LogLevel.Warn)

    sess_seed   = 0 # np.random.randint(0, 2**30)
    sess_seed_g = 1 # np.random.randint(0, 2**30)
    print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
mic_poses = np.array([[4., 2., 5.]])
# mic_poses = np.array([[11., 3., 5.]])

sail_poses_dict = {}
sail_size    = np.array([  4.,  2., 4.]) / 2.
sail_poses   = np.array([[ 4., 16., 2.5],
                         [ 4., 16., 7.5],
                         [10., 16., 2.5],
                         [10., 16., 7.5]])
# sail_poses   = np.array([[ 11., 16., 5]])

config = {
    "box_dim":     [22., 18., 10.],
    "mic_pos":     mic_poses[0],
    "speaker_pos": [18.,  2., 5.],
    # "speaker_pos": [11., 1.5, 5.],
    "speaker_radius": 1.0,

    "absorption": 0.5,
    "scattering": 0.5,

    "wav_bins":  mic_poses.shape[0],
    "time_bins": 10,
    "max_time":  0.2,

    "integrator": "prb_reparam_acoustic",
    "max_depth": 2,
    "spp": 2**22,
}

tf      = mi.ScalarTransform4f
box_dim = np.array(config['box_dim']) / 2.
time    = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

# config["max_depth"] = utils.estimate_max_depth(config["box_dim"], config["max_time"], 1.5)
print(f"max_depth = {config['max_depth']}")

In [ ]:
scene_dict = utils.shoebox_scene(**config)

scene_dict["integrator"]["skip_direct"] = True
# scene_dict["sensor"]["film"]["rfilter"] = {
#     "type": "gaussian",
#     "stddev": 0.1 * 343. * config["max_time"] / config["time_bins"],
# }

del scene_dict["sensor"]["microphoneA"]
for i, m in enumerate(mic_poses):
    scene_dict["sensor"][f"microphone_{i}"] = {
        "type": "microphone",
        "cos_cutoff": 0.0,
        "to_world": tf.look_at(
            origin=(m - box_dim),
            target=(np.mean(sail_poses, axis=0) - box_dim),
            up=[0, 0, 1]
        ),
        # "to_world": tf.translate(m - box_dim),
    }

sail_vertex_base = mi.Transform4f.scale(sail_size).rotate([1., 0., 0.], angle=90.) @ mi.Point3f(
    [-1., -1.,  1.,  1.],
    [-1.,  1., -1.,  1.],
    [ 0.,  0.,  0.,  0.]
)

for i, s in enumerate(sail_poses):
    sail_poses_dict[f"sail_{i}.vertex_positions"] = mi.Point3f(s - box_dim)
    scene_dict[f"sail_{i}"] = {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
        "bsdf": {
            "type": "acousticbsdf",
            "scattering": { "type": "spectrum", "value": 0.1 },
            "absorption": { "type": "spectrum", "value": 0.2 },
        },
        "to_world": tf.translate(s - box_dim).scale(sail_size).rotate([1., 0., 0.], angle=90.),
    }

### Optimization Setup

In [ ]:
opt = mi.ad.Adam(lr=0.0025)
# opt["sail_0.vertex_positions"] = mi.Point2f(0.0, 0.0)
# opt["sail_1.vertex_positions"] = mi.Point2f( 0.10, 0.7)
opt["sail_0.vertex_positions"] = mi.Point2f(-0.10, 0.2)
opt["sail_1.vertex_positions"] = mi.Point2f( 0.10, 0.2)
opt["sail_2.vertex_positions"] = mi.Point2f(-0.15, 0.5)
opt["sail_3.vertex_positions"] = mi.Point2f( 0.15, 0.5)

# V = np.load("../data/ceiling-sail/ceiling-sail-single-mic-01.npy")
# for i, key in enumerate(sail_poses_dict):
    # opt[key] = mi.Point2f(V[0, -1, i])
    # print(opt[key])
    # opt[key] = mi.Point2f(0.0, 0.0)
    # opt[key] = mi.Point2f(*(np.random.rand(2) * 2. - 1.))

def apply_transform(params_to_update, optim):
    for key in sail_poses_dict:
        optim[key] = dr.clamp(optim[key], -2., 2.)
        transf = mi.Transform4f.translate(sail_poses_dict[key]) \
                               .rotate(axis=[1., 0., 0.], angle=(optim[key].x * 30.)) \
                               .rotate(axis=[0., 0., 1.], angle=(optim[key].y * 30.))
        params_to_update[key] = dr.ravel(transf @ sail_vertex_base)
    params_to_update.update()

In [ ]:
def loss(hist, ref=None):
    assert ref is None
    return dr.sum(hist[:, :, 0])
    # return dr.mean(dr.sqr(hist[:, :, 0]))
    # return dr.rcp(dr.mean(hist[:, :, 0]))

### Visualization

In [ ]:
visualize = False
out_view = False
if visualize and out_view:
    scene_dict_vis = utils.shoebox_scene_visual(resf=6, **config)

    scene_dict_vis["sensor"]["film"]["width"]  = 200 * 4
    scene_dict_vis["sensor"]["film"]["height"] = 100 * 3
    scene_dict_vis["sensor"]["cameraA"]["to_world"] = tf.look_at(
        origin=np.array([-44., -1.8, 40.]),
        # origin=np.array([-32., 22, 40.]),
        target=np.zeros(3),
        up=[0, 1, 0]
    )
    scene_dict_vis["sensor"]["cameraB"] = {
        "type": "perspective",
        "to_world": tf.look_at(
            origin=np.array([0., -1.8, 55.]),
            target=np.zeros(3),
            up=[0, 1, 0]
        ),
    }

    del scene_dict_vis["microphoneA"]
    for i, m in enumerate(mic_poses):
        scene_dict_vis[f"microphone_{i}"] = {
            "type": "sphere",
            "radius": .25,
            "to_world": tf.translate(m - box_dim),
            "emitter": { "type": "area", "radiance": { "type": "rgb", "value": [.4, .0, .1] } },
        }

    sail_colors = [
        { "type": "rgb", "value": [0.8863, 0.2902, 0.2000] },
        { "type": "rgb", "value": [0.4667, 0.4667, 0.4667] },
        { "type": "rgb", "value": [0.9843, 0.7569, 0.3686] },
        { "type": "rgb", "value": [0.5569, 0.7294, 0.2588] }
    ]

    assert sail_poses.shape[0] <= len(sail_colors)

    for i, s in enumerate(sail_poses):
        scene_dict_vis[f"sail_{i}"] = {
            "type": "ply",
            "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
            "bsdf": { "type": "diffuse", "reflectance": sail_colors[i] },
            "to_world": tf.translate(s - box_dim).scale(sail_size).rotate([1., 0., 0.], angle=90.)
        }

In [ ]:
if visualize and not out_view:
    scene_dict_vis = utils.shoebox_scene_visual(resf=6, **config)

    scene_dict_vis["integrator"]["max_depth"] = 2

    del scene_dict_vis["microphoneA"]
    scene_dict_vis["speaker"]["emitter"]["radiance"]["value"] = 1e2

    scene_dict_vis["sensor"]["film"]["width"]  = 200 * 4
    scene_dict_vis["sensor"]["film"]["height"] = 200 * 3

    scene_dict_vis["sensor"]["cameraA"]["to_world"] = tf.look_at(
        origin=(mic_poses[0] - box_dim),
        target=(np.mean(sail_poses, axis=0) - box_dim),
        up=[0, 0, 1],
    )
    scene_dict_vis["sensor"]["cameraA"]["fov"] = 80

    for i, s in enumerate(sail_poses):
        scene_dict_vis[f"sail_{i}"] = scene_dict[f"sail_{i}"]

In [ ]:
if visualize:
    mi.set_variant("cuda_ad_rgb")
    scene  = mi.load_dict(scene_dict_vis)
    params = mi.traverse(scene)
    apply_transform(params, opt)
    img = mi.render(scene, seed=sess_seed)
    utils.plot_img(img)

In [ ]:
if False and visualize:
    # TODO rework
    # exi = 19
    V = np.load(f"../data/ceiling-sail/ceiling-sail-single-mic.npy")
    G = np.load(f"../data/ceiling-sail/ceiling-sail-single-mic_grads.npy") * -1.
    L = np.load(f"../data/ceiling-sail/ceiling-sail-single-mic_losses.npy")
    assert V.shape[0] == L.shape[0]

    L_min, L_max = np.min(L) * 0.99, np.max(L) * 1.01

    for i in trange(240): #V.shape[0]):
        opt["sail_0.vertex_positions"] = mi.Point2f(V[i, 0])
        opt["sail_1.vertex_positions"] = mi.Point2f(V[i, 1])
        opt["sail_2.vertex_positions"] = mi.Point2f(V[i, 2])
        opt["sail_3.vertex_positions"] = mi.Point2f(V[i, 3])
        apply_transform(params_vis)

        img = mi.render(scene_vis, seed=sess_seed)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))

        ax1.plot(L[:i+1])
        ax1.scatter(i, L[i])
        ax1.set_xlim(-5, L.shape[0] + 5)
        ax1.set_ylim(L_min, L_max)
        # ax1.set_yscale("log")
        ax1.set_xlabel("iteration")
        ax1.set_ylabel("loss")

        colors = ['C0', 'C3', 'C4', 'C5']
        for j, c in enumerate(colors):
            ax2.plot(V[:i+1, j, 0], V[:i+1, j, 1], c=c)
        ax2.quiver(V[i, :, 0], V[i, :, 1], G[i, :, 0], G[i, :, 1], color=colors)

        ticks = (np.linspace(-1., 1., 7), np.linspace(-30., 30., 7))
        ax2.set_xticks(*ticks)
        ax2.set_yticks(*ticks)
        ax2.set_xlim(-1.1, 1.1)
        ax2.set_ylim(-1.1, 1.1)
        ax2.set_xlabel("$R_x$ in degrees")
        ax2.set_ylabel("$R_z$ in degrees")

        fig.tight_layout()
        fig.canvas.draw()
        w, h = fig.canvas.get_width_height()
        plot = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
        plt.close()

        img_conv = mi.TensorXf(bitmap(img, convert=True)).numpy().astype(np.uint8)
        img_merg = np.vstack([img_conv, plot])
        mi.util.write_bitmap(f"../data/ceiling-sail/images/image_{i:03d}.png", img_merg)

### Load and Render Acoustic Scene

In [ ]:
scene = mi.load_dict(scene_dict)
params = mi.traverse(scene)
apply_transform(params, opt)

img = mi.render(scene, seed=sess_seed)
utils.plot_hist(img[:, :, 0], **config)

### Compare neutral to optimized position

In [ ]:
if False:
    for k in range(sail_poses.shape[0]):
        opt[f"sail_{k}.vertex_positions"] = mi.Point2f(0., 0.)

    apply_transform(params, opt)
    img_ref = mi.render(scene, seed=sess_seed)

    V = np.load("../data/ceiling-sail/ceiling-sail-single-mic.npy")
    for k in range(sail_poses.shape[0]):
        opt[f"sail_{k}.vertex_positions"] = mi.Point2f(V[-1, k])

    apply_transform(params, opt)
    img = mi.render(scene, seed=sess_seed)

    plot_hist(img[:, :, 0] - img_ref[:, :, 0], **config)

### Visualize Gradient-Spectogram

In [ ]:
apply_transform(params, opt)
img = mi.render(scene, seed=sess_seed, seed_grad=sess_seed_g)

dr.enable_grad(img)
l = loss(img)
dr.backward(l)
g = mi.TensorXf(dr.grad(img))

print(dr.min(g), dr.max(g))
utils.plot_hist(g[:, :, 0], **config)

### Main Optimization Loop

In [ ]:
%matplotlib ipympl

vals, losses, grads = [], [], []

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.set_title("losses")
ax1.set_xlim(-1, 51)

ax2.set_title("values")
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1);

In [ ]:
iters = 50
if iters > 1:
    n  = len(vals) + iters

# for i in trange(iters) if iters > 1 else range(iters):
for i in range(iters):
    apply_transform(params, opt)
    img = mi.render(scene, params, seed=sess_seed+i, seed_grad=sess_seed_g+i)

    f = -1.
    l = f * loss(img)
    dr.backward(l, flags=dr.ADFlag.ClearNone if iters < 2 else dr.ADFlag.Default)

    if iters < 2:
        # key = "sail_1.vertex_positions"
        # display(dr.unravel(mi.Point3f, dr.grad(params[key])))
        display(dr.grad(opt[key]))
    else:
        angles, sail_grads = [], []
        for key in sail_poses_dict:
            angles.append(opt[key])
            sail_grads.append(dr.grad(opt[key]))

        vals.append(angles)
        grads.append(sail_grads)
        losses.append(l)

        opt.step()

        V = np.array(vals)[:, :, 0]
        G = np.array(grads)[:, :, 0]

        ax1.clear()
        ax1.set_title("losses")
        ax1.set_xlim(-n * 0.02, n * 1.02)

        ax1.plot(np.array(losses)[:, 0])

        ax2.clear()
        ax2.set_title("values")
        ax2.plot(V[:, :, 0], V[:, :, 1])
        ax2.set_xlim(-0.8, 0.8)
        ax2.set_ylim(-0.8, 0.8)

        ax2.quiver( V[-1, :, 0],  V[-1, :, 1],
                   -G[-1, :, 0], -G[-1, :, 1])

        fig.canvas.draw()

In [ ]:
if iters > 1:
    V = np.stack([V, G])
    for key in sail_poses_dict:
        print(opt[key])

    # np.save("../data/ceiling-sail/ceiling-sail-single-mic-02.npy", V)

### Loss per optimization step

In [ ]:
if True:
    # fname = "../data/ceiling-sail/ceiling-sail-01.npy"
    # V = np.load(fname)
    losses = []

    for i in trange(V.shape[1]):
        opt["sail_0.vertex_positions"] = mi.Point2f(V[0, i, 0])
        opt["sail_1.vertex_positions"] = mi.Point2f(V[0, i, 1])
        opt["sail_2.vertex_positions"] = mi.Point2f(V[0, i, 2])
        opt["sail_3.vertex_positions"] = mi.Point2f(V[0, i, 3])
        apply_transform(params, opt)

        img = mi.render(scene, seed=sess_seed)
        losses.append(loss(img))

    # np.save(fname.replace('.npy', '_losses.npy'), np.array(losses))

    f, ax = plt.subplots(1, 1)
    ax.plot(np.array(losses)[:, 0])
    plt.show()

### Loss Heatmap

In [ ]:
heatmap = False

if heatmap:
    k = 0
    # opt["sail_0.vertex_positions"] = mi.Point2f(-0.1, 0.7)
    # opt["sail_1.vertex_positions"] = mi.Point2f( 0.1, 0.7)
    # opt["sail_2.vertex_positions"] = mi.Point2f(V[0, -1, 2])
    # opt["sail_3.vertex_positions"] = mi.Point2f(V[0, -1, 3])

    vA   = np.linspace(0.2, 0.35, 16, endpoint=True)
    vB   = np.linspace(0.15, 0.30, 16, endpoint=True)
    a, b = np.meshgrid(vA, vB)
    data = np.zeros((3,) + a.shape)
    print(vA)
    print(vB)

    for i in trange(a.shape[0]):
        for j in range(a.shape[1]):
            opt[f"sail_{k}.vertex_positions"] = mi.Point2f(a[i, j], b[i, j])
            apply_transform(params, opt)

            img = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g)
            l   = loss(img)
            dr.backward(l)
            data[0, i, j] = l[0]
            data[1, i, j] = dr.grad(opt[f"sail_{k}.vertex_positions"]).x[0]
            data[2, i, j] = dr.grad(opt[f"sail_{k}.vertex_positions"]).y[0]

In [ ]:
if heatmap:
    n = 2
    fig, ax = plt.subplots(1, 1, figsize=(7, 6))

    contf = ax.contourf(a, b, data[0], cmap="inferno")
    # ax.contour(a, b, data[0], colors='k', alpha=0.75, linewidths=0.3)

    ax.quiver(a, b, data[1], data[2], color='C1')

    fig.colorbar(contf, ax=ax, fraction=0.1, shrink=0.8).set_label('loss')
    # ax.set_xticks(vA[::n], np.round(vA * 30., 1)[::n])
    # ax.set_yticks(vB[::n], np.round(vB * 30., 1)[::n])
    ax.set_xlabel("$R_x$")
    ax.set_ylabel("$R_z$")
    ax.set_title(f"Loss Heatmap - ceiling sail {k}")

    # fig.savefig(f"../data/ceiling-sail/ceiling-sail-single-mic-heatmap-sail_{k}.png")
    fig.show()

### Average distance/time emitter-sail-receiver reflection

In [ ]:
em_pos       = np.array(config["speaker_pos"])
em_dist      = np.linalg.norm(sail_poses - em_pos,       axis=1)
mic1_dist    = np.linalg.norm(sail_poses - mic_poses[0], axis=1) + em_dist
em_mic1_dist = np.linalg.norm(em_pos     - mic_poses[0], axis=0)

print(mic1_dist    / 343)
print(em_mic1_dist / 343)